In [9]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

from utils import process_articles_directory

In [22]:
result = process_articles_directory("data/plaintext_articles")
print(f"There are : {len(result)} articles")
print(f"Example article: {result[0][0]}")
print(f"Related subjects: {result[0][1]}")
print("Descriptive paragraph: ")
for line in result[0][2].splitlines():
    for i in range(0, len(line), 100):  # Step through each line in chunks of 29 characters
        print(line[i:i+100])            # Print each 29-character chunk of the line
    print()  

There are : 4598 articles
Example article: Second Crusade
Related subjects: ['Military History and War', 'Religious disputes']
Descriptive paragraph: 
The Second Crusade was the second major crusade launched from Europe, called in 1145 in response to 
the fall of the County of Edessa the previous year. Edessa was the first of the Crusader states to h
ave been founded during the First Crusade (1095–1099), and was the first to fall. The Second Crusade
 was announced by Pope Eugene III, and was the first of the crusades to be led by European kings, na
mely Louis VII of France and Conrad III of Germany, with help from a number of other important Europ
ean nobles. The armies of the two kings marched separately across Europe and were somewhat hindered 
by Byzantine emperor Manuel I Comnenus; after crossing Byzantine territory into Anatolia, both armie
s were separately defeated by the Seljuk Turks. Louis and Conrad and the remnants of their armies re
ached Jerusalem and, in 1148, participate